In [ ]:
import ipyleaflet
import ogr, os
import pandas as pd
import geopandas as gpd
import shapefile
import json 
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl, basemaps, basemap_to_tiles
)

import pyproj
from shapely.ops import transform
from shapely.geometry import Polygon
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile

from IPython.html import widgets
from IPython.display import display
import osmnx as ox
from tkinter import filedialog
from tkinter import *
import time
from shapely.geometry import mapping, shape
from shapely.geometry import box as shBox
import fiona
import numpy as np
import rasterio
import gdal

root = Tk()
root.withdraw()
out = filedialog.askdirectory()


geo={'Open Street Maps':['North Carolina','South Carolina'],'MS Building Footprints':['North Carolina','South Carolina']}

print('Source and State of Interest Below:')


def print_State(State):
    print (State)

def select_State(Source):
    StateW.options = geo[Source]
    print(Source)

scW = widgets.Select(options=geo.keys())
init = scW.value
StateW = widgets.Select(options=geo[init])
j = widgets.interactive(print_State, State=StateW)
i = widgets.interactive(select_State, Source=scW)
display(i)
display(j)




     
rast=widgets.Checkbox(
    value=False,
    description='Check for raster files',
    disabled=False,
    indent=False
)

display(rast)




rO=widgets.BoundedIntText(
    value=30,
    min=0,
    max=1000,
    step=1,
    description='Res? (m):',
    disabled=False
)

display(rO)

print('Draw your coastal bounds region of interest in your chosen State within next cell. Rectangle tool recommended. Once drawing is complete, run the cell after to download your file. This may take a while as source files are large.')

In [ ]:
if StateW.value=='North Carolina':

    #ROI name
    roi="NorthCarolina"
    #Set buffer distance in decimal degreees
    dist=0.01


    watercolor = basemap_to_tiles(basemaps.Esri.WorldImagery, crs='espg:4326')

    m = Map(layers=(watercolor, ), center=(35,-75 ), zoom=7.7)


    dc = DrawControl(marker={'shapeOptions': {'color': '#0000FF'}},
                     rectangle={'shapeOptions': {'color': '#0000FF'}},
                     circle={'shapeOptions': {'color': '#0000FF'}},
                     circlemarker={},
                     )
    def handle_draw(target, action, geo_json):
        print(action)
        print(geo_json)

    dc.on_draw(handle_draw)
    m.add_control(dc)
    
if StateW.value=='South Carolina':
       #ROI name
    roi="SouthCarolina"
    osm_roi='South Carolina, USA'
    #Set buffer distance in decimal degreees
    dist=0.01


    watercolor = basemap_to_tiles(basemaps.Esri.WorldImagery, crs='espg:4326')

    m = Map(layers=(watercolor, ), center=(33,-79 ), zoom=7.7)


    dc = DrawControl(marker={'shapeOptions': {'color': '#0000FF'}},
                     rectangle={'shapeOptions': {'color': '#0000FF'}},
                     circle={'shapeOptions': {'color': '#0000FF'}},
                     circlemarker={},
                     )
    def handle_draw(target, action, geo_json):
        print(action)
        print(geo_json)

    dc.on_draw(handle_draw)
    m.add_control(dc)
    
    
m


In [ ]:
rasterchoice=str(rast.value)

if scW.value == 'MS Building Footprints':

    bnds= (dc.last_draw).get("geometry").get("coordinates")[0]
    bnds_poly=Polygon(bnds)
    bounds_gpd=gpd.GeoDataFrame(geometry=[bnds_poly])
    bounds_gpd=bounds_gpd.set_crs('EPSG:4326')

    print("Bounds shape created from drawing")

    coast=gpd.read_file('https://geodata.lib.berkeley.edu/download/file/stanford-xv279yj9196-geojson.json')

    print('US coastline saved')


    #clip coastal polyline
    roi_coast=gpd.clip(coast, bounds_gpd)
    roi_coast=roi_coast.to_crs('EPSG:4326')
    buffer=roi_coast.buffer(dist)

    print('US Coastline buffered by 10km and clipped to bounds area')


    foot_url='https://usbuildingdata.blob.core.windows.net/usbuildings-v2/'+roi+'.geojson.zip'
    footfile= urlopen(foot_url).read()
    footfile=BytesIO(footfile)
    foot_doc=ZipFile(footfile)
    foot_cont=foot_doc.open(roi+'.geojson')
    foot_gpd=gpd.read_file(foot_cont)
    json_clipped = gpd.clip(foot_gpd, buffer)
    json_clipped = gpd.clip(json_clipped, bounds_gpd)
    json_clipped.set_crs('EPSG:4326')
    json_clipped.to_file(out+roi+'Bldgs_MS.geojson', driver='GeoJSON')
    print('MS data clipped to coastal buffer roi and downloaded to folder')
    bldgpoly=out+'/'+roi+'Bldgs_MS.geojson'
    
    

if scW.value == 'Open Street Maps':
    
    bnds= (dc.last_draw).get("geometry").get("coordinates")[0]
    bnds_poly=Polygon(bnds)
    bounds_gpd=gpd.GeoDataFrame(geometry=[bnds_poly])
    bounds_gpd=bounds_gpd.set_crs('EPSG:4326')

    print("Bounds shape created from drawing")

    coast=gpd.read_file('https://geodata.lib.berkeley.edu/download/file/stanford-xv279yj9196-geojson.json')
    print('US coastline saved')
    #clip coastal polyline
    roi_coast=gpd.clip(coast, bounds_gpd)
    roi_coast=roi_coast.to_crs('EPSG:4326')
    buffer=roi_coast.buffer(dist)
    print('US Coastline buffered by 10km and clipped to bounds area')


    tags = {"building": True}
    north=bnds[1]
    north=north[1]
    south=bnds[0]
    south=south[1]
    west=bnds[0]
    west=west[0]
    east=bnds[2]
    east=east[0]

    gdf = ox.geometries_from_bbox(north, south, east, west, tags)
    gdf_proj = ox.project_gdf(gdf)
    # save as a geojson to your folder. Coordinate system set to EPSG 4326
    osmgdf_save = gdf.applymap(lambda x: str(x) if isinstance(x, list) else x)
    osmgdf_save.drop(labels="nodes", axis=1)
    osmgdf_save=osmgdf_save.drop(labels="nodes", axis=1)
    osm_gpd=gpd.GeoDataFrame(osmgdf_save)
    osm_gpd=osm_gpd.set_crs('EPSG:4326')
    json_clipped = gpd.clip(osm_gpd, buffer)
    json_clipped = gpd.clip(json_clipped, bounds_gpd)
    json_clipped.set_crs('EPSG:4326')
    json_clipped.to_file(out+'/'+roi+'Bldgs_OSM.geojson', driver="GPKG")


    print('OSM Building data clipped to coastal buffer roi and downloaded to folder')
    bldgpoly=out+'/'+roi+'Bldgs_OSM.geojson'

if rasterchoice=='False':
    print('Done!')

if rasterchoice=='True':


    reso=rO.value

    #create polygon from bound box
    plygdf=gpd.GeoDataFrame(bnds)
    plygdf['lon']= plygdf[0]
    plygdf['lat']= plygdf[1]
    lon=(plygdf['lon'].to_numpy())
    lat=(plygdf['lat'].to_numpy())


    polygon_geom = zip(lon, lat)

    polygon_geom = Polygon(zip(lon, lat))
    crs = {'init': 'epsg:4326'}
    polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])       
    print(polygon.geometry)


    polygon.to_file(out+'/'+roi+r'_bounds.geojson', driver="GPKG")


    #convert polygon to reference raster

    NoDataVal = -9999

    # Open the data source and read in the extent
    inPolygonShp = out+'/'+roi+r'_bounds.geojson'
    outputRaster=out+ r'/reference_raster.tif'
    inGridSize=float(reso)/110575      # converting meters GCS coordinates
    shpDS = ogr.Open(inPolygonShp)
    shpLayer = shpDS.GetLayer()

    # Create the destination data source
    xMin, xMax, yMin, yMax = shpLayer.GetExtent()
    xRes = int((xMax - xMin) / inGridSize)
    yRes = int((yMax - yMin) / inGridSize)
    rasterDS = gdal.GetDriverByName('GTiff').Create(outputRaster, xRes, yRes, 1,    gdal.GDT_Byte)

    # Define spatial reference
    rasterDS.SetProjection(shpLayer.GetSpatialRef().ExportToWkt())
    rasterDS.SetGeoTransform((xMin, inGridSize, 0, yMax, 0, -inGridSize))
    rBand = rasterDS.GetRasterBand(1)
    rBand.SetNoDataValue(NoDataVal)
    rBand.Fill(NoDataVal)

    err = gdal.RasterizeLayer(rasterDS, [1], shpLayer, options = ["ALL_TOUCHED=TRUE", "ATTRIBUTE=POLYNUMB"])

    #For some reason only sucessful when ran twice...:
    inGridSize=float(reso)/110575      # converting meters GCS coordinates
    shpDS = ogr.Open(inPolygonShp)
    shpLayer = shpDS.GetLayer()

    # Create the destination data source
    xMin, xMax, yMin, yMax = shpLayer.GetExtent()
    xRes = int((xMax - xMin) / inGridSize)
    yRes = int((yMax - yMin) / inGridSize)
    rasterDS = gdal.GetDriverByName('GTiff').Create(outputRaster, xRes, yRes, 1,    gdal.GDT_Byte)

    # Define spatial reference
    rasterDS.SetProjection(shpLayer.GetSpatialRef().ExportToWkt())
    rasterDS.SetGeoTransform((xMin, inGridSize, 0, yMax, 0, -inGridSize))
    rBand = rasterDS.GetRasterBand(1)
    rBand.SetNoDataValue(NoDataVal)
    rBand.Fill(NoDataVal)

    err = gdal.RasterizeLayer(rasterDS, [1], shpLayer, options = ["ALL_TOUCHED=TRUE", "ATTRIBUTE=POLYNUMB"])


    t1=time.time()






    # set the input layers and output locations

    ## We use the extent, projection, and cell alignment of this raster to generate the outputs.
    inputRaster=outputRaster
    inputBuildingsShape2= bldgpoly
    outputFolder=out

    res=inGridSize


    # set the output rasters (results)
    bldgMaxToWriteRaster     = outputFolder +'/'+roi+r'_bldgMax.tif'
    bldgMinToWriteRaster     = outputFolder +'/'+roi+r'_bldgMin.tif'
    bldgAveToWriteRaster     = outputFolder +'/'+roi+r'_bldgAve.tif'
    bldgSumToWriteRaster     = outputFolder +'/'+roi+r'_bldgSum.tif'
    bldgCountToWriteRaster   = outputFolder+ '/'+roi+ r'_bldgCount.tif'
    bldgCentroidCountToWriteRaster= outputFolder+'/'+ roi+ r'_bldgCentroidCount.tif'



    bldgSumDataType='float32'
    bldgMaxDataType='float64'
    bldgMinDataType='float64'
    bldgAvgDataType='float64'
    bldgCntDataType='int32'
    bldgCenteriodCntDataType='int32'




    with rasterio.open(inputRaster) as nlcdWindow:
        kwds = nlcdWindow.meta.copy()
        thisWindowWidth=nlcdWindow.width
        thisWindowHeight=nlcdWindow.height
        cellSize= kwds['transform'][0]
        topY  = nlcdWindow.bounds[3]
        leftX = nlcdWindow.bounds[0]
        windowShape = nlcdWindow.shape
        topLeftCornerWindow = [nlcdWindow.bounds[0],nlcdWindow.bounds[3]]
        profile = nlcdWindow.profile



    # start the arrays for each raster output. the rasters have the same shape as the nlcd window
    arBldgSum=np.zeros(windowShape)
    arBldgSumAllBldgArea=np.zeros(windowShape)
    arBldgMax=np.zeros(windowShape)
    arBldgMin=np.zeros(windowShape)
    arBldgCount=np.zeros(windowShape)
    arBldgCentroid=np.zeros(windowShape)



    # read the building shape file as geometries and put them in a list
    buildingsPolys = [shape(bldg['geometry']) for bldg in fiona.open(inputBuildingsShape2)]

    print ('read the buildings',(time.time()-t1))

    # an error variable to know howw many intersection fail
    er=0
    # grab each building in this loop
    for bldg in buildingsPolys:
        # get the area of the building
        bldgArea=bldg.area
        # get the bound of the building
        bldgBounds=bldg.bounds

        # calculate the count of centroid for each cell
        # to make sure the point is inside the polygon this method can be used instead: bldg.representative_point()
        centroidX=(bldg.centroid.coords)[0][0]
        centroidY=(bldg.centroid.coords)[0][1]
        col   = int((centroidX - leftX)/cellSize)
        row   = int((topY - centroidY)/cellSize)
        arBldgCentroid[row,col]+=1

        # let's find the distance form the top left corner of the raster
        xDifTL=bldgBounds[0]-topLeftCornerWindow[0]
        yDifTL=topLeftCornerWindow[1]-bldgBounds[3]
        # lets find the top left corner of the first grid cell that is supposed to be intersected later
        colStart=int(xDifTL//res)
        rowStart=int(yDifTL//res)

        # lets find the end row and collumn of the window cells of this building
        colEnd=int((bldgBounds[2]-topLeftCornerWindow[0])//res)+1
        rowEnd=int((topLeftCornerWindow[1]-bldgBounds[1])//res)+1

        # get the shape of the window cell that has intersection with this building
        nmRowWinCell=rowEnd-rowStart
        nmColWinCell=colEnd-colStart
        shapeWinCell=[nmRowWinCell,nmColWinCell]


        # let's go through each cell in the window cells whcih are the cells having intersection with
        # this specific budiling
        for row1 in range(0, shapeWinCell[0]):
            for col1 in range(0,shapeWinCell[1]):
                # the current cell
                thisRow=row1+rowStart
                thisCol=col1+colStart
                #print (thisRow,thisCol)
                #shapely.geometry.box(minx, miny, maxx, maxy, ccw=True)
                # here create a shape polygon geometry of the current cell
                minXBox=thisCol*res+topLeftCornerWindow[0]
                maxYBox=topLeftCornerWindow[1]-(thisRow*res)
                maxXBox=minXBox+res
                minYBox=maxYBox-res
                thisCell=shBox(minXBox, minYBox, maxXBox, maxYBox, ccw=True)
                # run the intersection
                if bldg.intersects(thisCell) ==True:
                    try:
                        area = bldg.intersection(thisCell).area
                        #print(area,bldgArea,(thisRow,thisCol))
                        # update the arrays with the values being generated through intersection
                        # count
                        arBldgCount[thisRow,thisCol]=arBldgCount[thisRow,thisCol]+1
                        # sum
                        arBldgSum[thisRow,thisCol]=arBldgSum[thisRow,thisCol]+area
                        # sums of buildings to build average building area later
                        arBldgSumAllBldgArea[thisRow,thisCol]=arBldgSumAllBldgArea[thisRow,thisCol]+bldgArea
                        # Max
                        if arBldgMax[thisRow,thisCol]<area:
                            arBldgMax[thisRow,thisCol]=bldgArea
                        # min
                        if arBldgMin[thisRow,thisCol]==0 or arBldgMin[thisRow,thisCol]>area:
                            arBldgMin[thisRow,thisCol]=bldgArea

                    except:
                        er+=1

    # calcualate the average building array using sum of building areas and count
    arAveBldgArea= np.divide(arBldgSumAllBldgArea,arBldgCount,where=arBldgCount!=0)
    print('error is', er)
    print ('built all arrays',(time.time()-t1))




    arBldgSum = (arBldgSum).astype(bldgSumDataType)
    arAveBldgArea = (arAveBldgArea).astype(bldgAvgDataType)
    arBldgMax= (arBldgMax).astype(bldgMaxDataType)
    arBldgMin=(arBldgMin).astype(bldgMinDataType)
    arBldgCount=arBldgCount.astype(bldgCntDataType)
    arBldgCentroid=arBldgCentroid.astype(bldgCenteriodCntDataType)

    # write arrays to rasters



    profile['dtype']=bldgMaxDataType
    with rasterio.open(bldgMaxToWriteRaster, 'w', **profile) as output:
        output.write_band(1,arBldgMax)

    profile['dtype']=bldgMinDataType    
    with rasterio.open(bldgMinToWriteRaster, 'w', **profile) as output:
        output.write_band(1,arBldgMin)

    profile['dtype']=bldgAvgDataType    
    with rasterio.open(bldgAveToWriteRaster, 'w', **profile) as output:
        output.write_band(1,arAveBldgArea)

    profile['dtype']=bldgSumDataType    
    with rasterio.open(bldgSumToWriteRaster, 'w', **profile) as output:
        output.write_band(1,arBldgSum)

    profile['dtype']=bldgCntDataType    
    with rasterio.open(bldgCountToWriteRaster, 'w', **profile) as output:
        output.write_band(1,arBldgCount)


    profile['dtype']=bldgCenteriodCntDataType    
    with rasterio.open(bldgCentroidCountToWriteRaster, 'w', **profile) as output:
        output.write_band(1,arBldgCentroid)
    print ('Building Rasters Created')